In [8]:
import os
import pandas as pd



In [13]:
processed_shutoffs = pd.read_csv(
    '../data/processed/processed-shutoffs.csv', index_col=0, dtype=str)
print(processed_shutoffs.head(3))
#pd.to_datetime
weather = pd.DataFrame(columns = ['circuit_name', 'deenergize_time', 'tmin_d-4', 'tmin_d-3', 'tmin_d-2', 
'tmin_d-1', 'tmin_d-0', 'tmax_d-4', 'tmax_d-3', 'tmax_d-2','tmax_d-1', 'tmax_d-0', 'wspd_d-4', 
'wspd_d-3', 'wspd_d-2', 'wspd_d-1', 'wspd_d-0'])

#os.listdir() to iterate through files

# shutoffs_w_weather = pd.join(processed_shutoffs, weather, on = "circuit_name", deenergize_time)

# shutoffs_w_weather.to_csv('../data/processed/processed-shutoffs-weather.csv', index=False)


                     deenergize_time               restoration_time  \
circuit_name                                                          
APPLE HILL-1103  2018-10-14 21:00:00  2018-10-15 16:29:00.000000000   
APPLE HILL-1104  2018-10-14 21:17:00  2018-10-15 18:37:00.000000000   
APPLE HILL-2102  2018-10-14 21:05:00  2018-10-16 18:04:00.000000000   

                time_out_min  \
circuit_name                   
APPLE HILL-1103       1169.0   
APPLE HILL-1104       1280.0   
APPLE HILL-2102       2699.0   

                                                   key_communities hftd_tier  \
circuit_name                                                                   
APPLE HILL-1103                                Camino, Placerville       3.0   
APPLE HILL-1104                Camino, Placerville, Pollock Pines,       3.0   
APPLE HILL-2102  Camino, Grizzly Flats, Mount Aukum, Placervill...       3.0   

                total_affected residential_affected zip_code  \
circuit_name      